In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score




os.getcwd()
os.chdir('/Users/ting/Desktop/kaggle_Zillow_Prize/')

讀取資料

In [2]:
train = pd.read_csv('train_2017.csv')
prop = pd.read_csv('properties_2017.csv')
samp = pd.read_csv('sample_submission.csv')

/Users/ting/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


lightgbm

In [3]:
#set training data
train_t = train.merge(prop, how = 'left', on = 'parcelid')
train_x = train_t.drop(['logerror'], axis=1)
train_y = train_t['logerror'].values

In [4]:
#尋找na超過95%的欄位
train_x_test_nanu = train_x.fillna(0)
dr_na_cl = []
for i in train_x_test_nanu.dtypes.index.values:
    if (len(train_x_test_nanu.ix[train_x_test_nanu.ix[:, i] == 0, i]))/len(train_x_test_nanu[[i]]) > 0.95:
        print(i)
        dr_na_cl.append(i)
dr_na_cl

/Users/ting/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  """


architecturalstyletypeid
basementsqft
buildingclasstypeid
decktypeid
finishedsquarefeet13
finishedsquarefeet15
finishedsquarefeet6
hashottuborspa
poolsizesum
pooltypeid10
pooltypeid2
storytypeid
typeconstructiontypeid
yardbuildingsqft17
yardbuildingsqft26
fireplaceflag
taxdelinquencyflag
taxdelinquencyyear


['architecturalstyletypeid',
 'basementsqft',
 'buildingclasstypeid',
 'decktypeid',
 'finishedsquarefeet13',
 'finishedsquarefeet15',
 'finishedsquarefeet6',
 'hashottuborspa',
 'poolsizesum',
 'pooltypeid10',
 'pooltypeid2',
 'storytypeid',
 'typeconstructiontypeid',
 'yardbuildingsqft17',
 'yardbuildingsqft26',
 'fireplaceflag',
 'taxdelinquencyflag',
 'taxdelinquencyyear']

In [5]:
#刪除空直超過95%的欄位
train_x = train_x.drop(dr_na_cl, axis = 1)

In [6]:
#找出objects的欄位
train_x.dtypes[train_x.dtypes == 'object'].index.values
#array(['transactiondate', 'propertycountylandusecode', 'propertyzoningdesc'], dtype=object)
#'transactiondate'看實際值，不把他分類

#處理propertycountylandusecode 跟此欄位相關欄位也是空值，所以無法借用來處理
train_x['propertycountylandusecode'].isnull = 'None'

#處理propertyzoningdesc'
pd.DataFrame(train_x.loc[train_x['propertyzoningdesc'].isnull(), 'propertylandusetypeid']).to_csv('propertylandusetypeid_col_uni.csv')
train_x.loc[train_x['propertyzoningdesc'].isnull(), 'propertylandusetypeid'].value_counts()
#246~8 260 261 263 265 266 269 275
#246
train_x.loc[train_x['propertylandusetypeid'] == 246, 'propertyzoningdesc'].value_counts()
#LAR2
train_x[train_x['propertyzoningdesc'].isnull()].loc[train_x.loc[train_x['propertyzoningdesc'].isnull(), 'propertylandusetypeid'] == 246, 'propertyzoningdesc']  = 'LAR2'
#247
train_x.loc[train_x['propertylandusetypeid'] == 247, 'propertyzoningdesc'].value_counts()
#LAR2
train_x[train_x['propertyzoningdesc'].isnull()].loc[train_x.loc[train_x['propertyzoningdesc'].isnull(), 'propertylandusetypeid'] == 247, 'propertyzoningdesc']  = 'LAR2'
#248
train_x.loc[train_x['propertylandusetypeid'] == 248, 'propertyzoningdesc'].value_counts()
#LARD1.5
train_x[train_x['propertyzoningdesc'].isnull()].loc[train_x.loc[train_x['propertyzoningdesc'].isnull(), 'propertylandusetypeid'] == 248, 'propertyzoningdesc']  = 'LARD1.5'
#260
train_x.loc[train_x['propertylandusetypeid'] == 260, 'propertyzoningdesc'].value_counts()
train_x[train_x['propertyzoningdesc'].isnull()].loc[train_x.loc[train_x['propertyzoningdesc'].isnull(), 'propertylandusetypeid'] == 260, 'propertyzoningdesc']  = 'LAR3'
#261
train_x.loc[train_x['propertylandusetypeid'] == 261, 'propertyzoningdesc'].value_counts()
train_x[train_x['propertyzoningdesc'].isnull()].loc[train_x.loc[train_x['propertyzoningdesc'].isnull(), 'propertylandusetypeid'] == 261, 'propertyzoningdesc']  = 'LAR1'
#263
train_x.loc[train_x['propertylandusetypeid'] == 263, 'propertyzoningdesc'].value_counts()
train_x[train_x['propertyzoningdesc'].isnull()].loc[train_x.loc[train_x['propertyzoningdesc'].isnull(), 'propertylandusetypeid'] == 263, 'propertyzoningdesc']  = 'LCA21*' 
#265
train_x.loc[train_x['propertylandusetypeid'] == 265, 'propertyzoningdesc'].value_counts()
train_x[train_x['propertyzoningdesc'].isnull()].loc[train_x.loc[train_x['propertyzoningdesc'].isnull(), 'propertylandusetypeid'] == 265, 'propertyzoningdesc']  = 'None' 
#266
train_x.loc[train_x['propertylandusetypeid'] == 266, 'propertyzoningdesc'].value_counts()
train_x[train_x['propertyzoningdesc'].isnull()].loc[train_x.loc[train_x['propertyzoningdesc'].isnull(), 'propertylandusetypeid'] == 266, 'propertyzoningdesc']  = 'LAR3' 
#269
train_x.loc[train_x['propertylandusetypeid'] == 269, 'propertyzoningdesc'].value_counts()
train_x[train_x['propertyzoningdesc'].isnull()].loc[train_x.loc[train_x['propertyzoningdesc'].isnull(), 'propertylandusetypeid'] == 269, 'propertyzoningdesc']  = 'SCUR2' 
#275
train_x.loc[train_x['propertylandusetypeid'] == 275, 'propertyzoningdesc'].value_counts()
train_x[train_x['propertyzoningdesc'].isnull()].loc[train_x.loc[train_x['propertyzoningdesc'].isnull(), 'propertylandusetypeid'] == 275, 'propertyzoningdesc']  = 'None' 


/Users/ting/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [14]:
#用0填充欄位,get dummies
train_x = train_x.fillna(0)
train_x_tra = train_x.transactiondate
train_x = train_x.drop('transactiondate', axis=1)
train_x = pd.get_dummies(train_x)
train_x['transactiondate'] = train_x_tra
train_x

,parcelid,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet50,...,propertyzoningdesc_WVRPD12U*,propertyzoningdesc_WVRPD12U-R,propertyzoningdesc_WVRPD17U*,propertyzoningdesc_WVRPD18U*,propertyzoningdesc_WVRPD40000,propertyzoningdesc_WVRPD4OOOO,propertyzoningdesc_WVRPD56*,propertyzoningdesc_WVRR,propertyzoningdesc_WVRR1-RPD1,transactiondate
0,14297519,0.0,3.5,4.0,0.0,3.5,0.0,3100.0,3100.0,0.0,...,0,0,0,0,0,0,0,0,0,2017-01-01
1,17052889,0.0,1.0,2.0,0.0,1.0,1465.0,1465.0,1465.0,1465.0,...,0,0,0,0,0,0,0,0,0,2017-01-01
2,14186244,0.0,2.0,3.0,0.0,2.0,0.0,1243.0,1243.0,0.0,...,0,0,0,0,0,0,0,0,0,2017-01-01
3,12177905,0.0,3.0,4.0,8.0,3.0,0.0,2376.0,2376.0,0.0,...,0,0,0,0,0,0,0,0,0,2017-01-01
4,10887214,1.0,3.0,3.0,8.0,3.0,0.0,1312.0,1312.0,0.0,...,0,0,0,0,0,0,0,0,0,2017-01-01
5,17143294,0.0,2.0,3.0,0.0,2.0,842.0,1492.0,1492.0,842.0,...,0,0,0,0,0,0,0,0,0,2017-01-01
6,12095076,1.0,3.0,4.0,9.0,3.0,0.0,2962.0,2962.0,0.0,...,0,0,0,0,0,0,0,0,0,2017-01-01
7,12069064,0.0,1.0,2.0,5.0,1.0,0.0,738.0,738.0,0.0,...,0,0,0,0,0,0,0,0,0,2017-01-01
8,12790562,0.0,3.0,4.0,9.0,3.0,0.0,3039.0,3039.0,0.0,...,0,0,0,0,0,0,0,0,0,2017-01-02
9,11542646,0.0,3.0,2.0,8.0,3.0,0.0,1290.0,1290.0,0.0,...,0,0,0,0,0,0,0,0,0,2017-01-02


In [16]:
train_col = train_x.columns
train_col

Index(['parcelid', 'airconditioningtypeid', 'bathroomcnt', 'bedroomcnt',
       'buildingqualitytypeid', 'calculatedbathnbr',
       'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet',
       'finishedsquarefeet12', 'finishedsquarefeet50',
       ...
       'propertyzoningdesc_WVRPD12U*', 'propertyzoningdesc_WVRPD12U-R',
       'propertyzoningdesc_WVRPD17U*', 'propertyzoningdesc_WVRPD18U*',
       'propertyzoningdesc_WVRPD40000', 'propertyzoningdesc_WVRPD4OOOO',
       'propertyzoningdesc_WVRPD56*', 'propertyzoningdesc_WVRR',
       'propertyzoningdesc_WVRR1-RPD1', 'transactiondate'],
      dtype='object', length=2024)

In [17]:
#抽出時間資訊
def get_da_fea(df):
    df['transactiondate'] = pd.to_datetime(df['transactiondate'])
    df['transactiondate_year'] = df['transactiondate'].dt.year
    df['transactiondate_year'] = df['transactiondate'].dt.month
    df['transactiondate_year'] = df['transactiondate'].dt.quarter
    return df


In [21]:
#準備數據
train_x = get_da_fea(train_x)
train_x['transactiondate'] = pd.DataFrame(np.float32(train_x['transactiondate']))
train_y= pd.DataFrame(train_y)
reg = LinearRegression()
reg.fit(train_x, train_y)

/Users/ting/anaconda/lib/python3.6/site-packages/scipy/linalg/basic.py:1018: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [30]:
#計算精準度
np.exp((train_y - pd.DataFrame(reg.predict(train_x))).mean())
#accuracy_score(train_y, pd.DataFrame(reg.predict(train_x)))

0    1.0
dtype: float64

In [31]:
#設置test data
samp['parcelid'] = samp['ParcelId']
test_x = samp.merge(prop, on='parcelid', how='left')
test_x

,ParcelId,201610,201611,201612,201710,201711,201712,parcelid,airconditioningtypeid,architecturalstyletypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,0,0,0,0,0,0,10754147,NaN,NaN,...,NaN,NaN,NaN,9.0,2016.0,9.0,NaN,NaN,NaN,NaN
1,10759547,0,0,0,0,0,0,10759547,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,0,0,0,0,0,0,10843547,NaN,NaN,...,1.0,NaN,660680.0,1434941.0,2016.0,774261.0,20800.37,NaN,NaN,NaN
3,10859147,0,0,0,0,0,0,10859147,NaN,NaN,...,1.0,NaN,580059.0,1174475.0,2016.0,594416.0,14557.57,NaN,NaN,NaN
4,10879947,0,0,0,0,0,0,10879947,NaN,NaN,...,1.0,NaN,196751.0,440101.0,2016.0,243350.0,5725.17,NaN,NaN,NaN
5,10898347,0,0,0,0,0,0,10898347,NaN,NaN,...,1.0,NaN,179072.0,287634.0,2016.0,108562.0,3661.28,NaN,NaN,NaN
6,10933547,0,0,0,0,0,0,10933547,NaN,NaN,...,NaN,NaN,404013.0,563029.0,2016.0,159016.0,6773.34,NaN,NaN,NaN
7,10940747,0,0,0,0,0,0,10940747,NaN,NaN,...,1.0,NaN,103553.0,698984.0,2016.0,595431.0,7857.84,NaN,NaN,NaN
8,10954547,0,0,0,0,0,0,10954547,NaN,NaN,...,NaN,NaN,NaN,9.0,2016.0,9.0,NaN,NaN,NaN,NaN
9,10976347,0,0,0,0,0,0,10976347,NaN,NaN,...,1.0,NaN,221771.0,265184.0,2016.0,43413.0,4054.76,NaN,NaN,NaN


In [32]:
#刪除空直超過95%的欄位
test_x = test_x.drop(dr_na_cl, axis = 1)

In [33]:
#處理obj欄位
test_x['propertycountylandusecode'].isnull = 'None'

test_x[test_x['propertyzoningdesc'].isnull()].loc[test_x.loc[test_x['propertyzoningdesc'].isnull(), 'propertylandusetypeid'] == 246, 'propertyzoningdesc']  = 'LAR2'

test_x[test_x['propertyzoningdesc'].isnull()].loc[test_x.loc[test_x['propertyzoningdesc'].isnull(), 'propertylandusetypeid'] == 247, 'propertyzoningdesc']  = 'LAR2'

test_x[test_x['propertyzoningdesc'].isnull()].loc[test_x.loc[test_x['propertyzoningdesc'].isnull(), 'propertylandusetypeid'] == 248, 'propertyzoningdesc']  = 'LARD1.5'

test_x[test_x['propertyzoningdesc'].isnull()].loc[test_x.loc[test_x['propertyzoningdesc'].isnull(), 'propertylandusetypeid'] == 260, 'propertyzoningdesc']  = 'LAR3'

test_x[test_x['propertyzoningdesc'].isnull()].loc[test_x.loc[test_x['propertyzoningdesc'].isnull(), 'propertylandusetypeid'] == 261, 'propertyzoningdesc']  = 'LAR1'

test_x[test_x['propertyzoningdesc'].isnull()].loc[test_x.loc[test_x['propertyzoningdesc'].isnull(), 'propertylandusetypeid'] == 263, 'propertyzoningdesc']  = 'LCA21*' 

test_x[test_x['propertyzoningdesc'].isnull()].loc[test_x.loc[test_x['propertyzoningdesc'].isnull(), 'propertylandusetypeid'] == 265, 'propertyzoningdesc']  = 'None' 

test_x[test_x['propertyzoningdesc'].isnull()].loc[test_x.loc[test_x['propertyzoningdesc'].isnull(), 'propertylandusetypeid'] == 266, 'propertyzoningdesc']  = 'LAR3' 

test_x[test_x['propertyzoningdesc'].isnull()].loc[test_x.loc[test_x['propertyzoningdesc'].isnull(), 'propertylandusetypeid'] == 269, 'propertyzoningdesc']  = 'SCUR2' 

test_x[test_x['propertyzoningdesc'].isnull()].loc[test_x.loc[test_x['propertyzoningdesc'].isnull(), 'propertylandusetypeid'] == 275, 'propertyzoningdesc']  = 'None' 


/Users/ting/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [ ]:
#用0填充欄位,get dummies
test_x = test_x.fillna(0)
test_x = pd.get_dummies(test_x)

In [ ]:
#用0填充欄位,get dummies
test_x = test_x.fillna(0)
test_x = pd.get_dummies(test_x)
test_x['transactiondate'] = '2016-10-01'
#['2016-10-01','2016-11-01','2016-12-01','2017-10-01','2017-11-01','2017-12-01']

In [ ]:
test_x = get_da_fea(test_x)
test_x['transactiondate'] = pd.DataFrame(np.float32(test_x['transactiondate']))
test_x= pd.DataFrame(test_x)

In [ ]:
lgb_pre = lgn_clf.predict(test_x)

In [ ]:
pd.DataFrame(lgb_pre).to_csv('lgb_pre201610.csv')